In [1]:
#libraries
import numpy as np
import random
import math
import cv2
import Queue
from collections import defaultdict
from collections import deque
import time

In [2]:
####ALL FUNCTIONS GO HERE####

#####FUNCTIONS FOR PATHFINDING######

# maze to graph converts 2d array to a graph
# taken from http://bryukh.com/labyrinth-algorithms/
def maze2graph(maze):
    
    print "Started maze2graph"
    startTime = time.time()
    
    height = len(maze)
    width = len(maze[0]) if height else 0
    graph = {(i, j): [] for j in range(width) for i in range(height) if not maze[i][j]}
    for row, col in graph.keys():
        if row < height - 1 and not maze[row + 1][col]:
            graph[(row, col)].append(("S", (row + 1, col)))
            graph[(row + 1, col)].append(("N", (row, col)))
        if col < width - 1 and not maze[row][col + 1]:
            graph[(row, col)].append(("E", (row, col + 1)))
            graph[(row, col + 1)].append(("W", (row, col)))
            
    print "Done maze2graph, took", time.time() - startTime, "seconds."
    
    return graph

#findpathbfs uses bredth first search to find a path from start to finish
# taken from http://bryukh.com/labyrinth-algorithms/
def find_path_bfs(maze, start, goal):
    
    print "Started find_path_bfs"
    startTime = time.time()
    
    queue = deque([("", start)])
    visited = set()
    graph = maze2graph(maze)
    while queue:
        path, current = queue.popleft()
        if current == goal:
            
            print "Done find_path_bfs, took", time.time() - startTime, "seconds."
            
            return path
        if current in visited:
            continue
        visited.add(current)
        for direction, neighbour in graph[current]:
            queue.append((path + direction, neighbour))
    return "NO WAY!"

#drawPath takes in an image and draws the path on it
def drawPath(image, direct, start):
    
    #creates blank image
    image = np.zeros((image.shape[0], image.shape[1], 3),np.uint8)
    
    colour = [random.randint(0,255),random.randint(0,255),random.randint(0,255)]
    
    current = (start[0]+ 100, start[1] + 200)
    
    for i in range(0, len(direct)):
        image[current] = colour
        
        #colourSpot(image, current[0], current[1], colour)
        
        if direct[i] == "N":
            current = (current[0]-1,current[1])
        if direct[i] == "S":
            current = (current[0]+1,current[1])
        if direct[i] == "E":
            current = (current[0],current[1]+1)
        if direct[i] == "W":
            current = (current[0],current[1]-1)
            
    return image

# creates maze using noise, then finds a path between two random points 
# and returns the path on a black bg
def generatePath(img):
    
    print "Started generatePath"
    startTime = time.time()

    #creates an array of noise, image is smaller than whole to save time/centre line
    maze = np.zeros((img.shape[0] - 200 , img.shape[1] - 400))

    for row in range(0, maze.shape[0]):
        for col in range(0, maze.shape[1]):
             maze[row][col] = random.choice([0, 0, 1])
                
    print img.shape            
    print maze.shape
    
    
    #creates points in maze
    start = (random.randint(0, maze.shape[0]-1), random.randint(0, maze.shape[1]-1))
    end = (random.randint(0, maze.shape[0]-1), random.randint(0 , maze.shape[1]-1))

    #makes sure that start and end aren't in obsticles
    while maze[start] or maze[end] == 1:
        start = (random.randint(0, maze.shape[0]-1), random.randint(0, maze.shape[1]-1))
        end = (random.randint(0, maze.shape[0]-1), random.randint(0 , maze.shape[1]-1))

    #creates and fills an array with the path
    direct = []
    direct = find_path_bfs(maze, start, end)

    #draws the path on a blank picture
    line = drawPath(img, direct, start)
    
    print "Done generatePath, took", time.time() - startTime, "seconds."
    
    return line

#### OTHER FUNCTONS ######

# puts a noise filter over a image and returns the result
def drugifyFast(image):
    
    print "Started drugifyFast"
    startTime = time.time()
    
    values = [[0,0,0],[0,0,255],[0,255,0],[0,255,255],[255,0,0],[255,0,255],[255,255,0],[255,255,255]]
    
    noise = np.zeros((image.shape[0],image.shape[1],3),np.uint8)
    
    for row in range(0, noise.shape[0]):
        for col in range(0, noise.shape[1]):
                noise[row][col] = values[random.randint(0,7)]
                
    image = cv2.addWeighted(noise, .65, image, .35, 0)
    
    print "Done drugifyFast, took", time.time() - startTime, "seconds."
    
    return image

#colours a certain pixel area around a certain point
def colourSpot(image, row, col, colour):
    for i in range(-1, 2):
        for j in range(-1, 2):
            image[row+i, col+j] = colour
        
#fades the right and left of a border higher num is less dark
def fadeEdges(image):
    
    startTime = time.time()
    
    print "Started fadeEdges"
    
    centre = image.shape[1]//2
    
    for i in range(0, image.shape[1]):
        dist = abs(centre - i) + 1
        val = dist/2.7
        
        if dist < 30:
            val = 15
        
        col = image[:,i,:]
        
        image[:,i,:] = cv2.addWeighted(col, 1, col, 0, -val)
                    
    print "Done fadeEdges, took", time.time() - startTime, "seconds."

    return image


In [3]:
def filterFrame(img):
    print "Started imageProcess"
    Time = time.time()

    img = drugifyFast(img)

    line = generatePath(img)

    img = cv2.addWeighted(img, 1, line, 1, -30)

    img = fadeEdges(img)

    print "Done imageProcess, took", time.time() - Time, "seconds in total."
    
    return img

img = cv2.imread('frame.jpg')

img = filterFrame(img)

cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Started imageProcess
Started drugifyFast


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#start of video editing

cap = cv2.VideoCapture('apartment.avi')

count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = filterFrame(frame)

        cv2.imwrite('vid' + str(count) + '.png',frame)
        count+=1
        
        #cv2.imshow('frame',frame)
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break
    else:
        break

# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()